In [142]:
import tensorflow as tf

In [143]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_generator = ImageDataGenerator()

In [144]:
directory_generator = img_generator.flow_from_directory(
    directory = r"dataset\data_sample\train", # the folder where the class subfolders can be found
    target_size = (512,512), # The (height,width) of the produced images
    class_mode = "sparse", # Wether the target should be represented by an index or a dummy vector
    batch_size=16, # The batch size of the produced batches
    shuffle = True #Whether to shuffle after all files have been selected once
    #subset = "training"
)

directory_generator_val = img_generator.flow_from_directory(
    directory = r"dataset\data_sample\test", # the folder where the class subfolders can be found
    target_size = (512,512), # The (height,width) of the produced images
    class_mode = "sparse", # Wether the target should be represented by an index or a dummy vector
    batch_size=16, # The batch size of the produced batches
    shuffle = True #Whether to shuffle after all files have been selected once
    #subset = "validation"
)

Found 3472 images belonging to 4 classes.
Found 1162 images belonging to 4 classes.


In [145]:
imgs, targets = next(iter(directory_generator))

In [146]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

modelconv = tf.keras.Sequential(
    [
     Conv2D(
         filters = 32,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu",
         input_shape = (imgs.shape[1], imgs.shape[2], imgs.shape[3]) # the input shape (height, width, channels)
     ),
     Conv2D(
         filters = 64,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu"
     ),
     Conv2D(
         filters = 64,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu"
     ),
     Conv2D(
         filters = 128,
         kernel_size = (3,3),
         strides = 2,
         padding = "same",
         activation = "relu"
     ),
    tf.keras.layers.Flatten(), # this layer turns multi-dimensional images into flat objects
    tf.keras.layers.Dense(24, activation="softmax")
    ]
)

In [147]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam

modelconv.compile(optimizer=Adam(),
              loss=SparseCategoricalCrossentropy(),
              metrics=[SparseCategoricalAccuracy()])

modelconv.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_40 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 24)             │     3,145,752 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,275,928 (12.50 MB)

 Trainable params: 3,275,928 (12.50 MB)

 Non-trainable params: 0 (0.00 B)

In [148]:
print(directory_generator.directory)

dataset\data_sample\train


In [ ]:
modelconv.fit(directory_generator, epochs = 10, validation_data=directory_generator_val)

In [ ]:
from plotly import graph_objects as go
color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]

history = modelconv.history
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'    
)
fig.show()